<a href="https://colab.research.google.com/github/sheon-j/nlp-study/blob/main/tensorflow-nlp-tutorial/week05_recurrent_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Study: Week 5 - Recurrent Neural Network

[딥 러닝을 이용한 자연어 처리 입문](https://wikidocs.net/book/2155) 스터디

---

**Contents**
1. 순환 신경망(Recurrent Neural Network)
2. 장단기 메모리(Long Short-Term Memory, LSTM)

## 1. 순환 신경망(Recurrent Neural Network)

* 피드 포워드 신경망의 한계
  * 피드 포워드 신경망은 입력의 길이가 고정되어 자연어 처리에 한계
  * 다양한 길이의 입력 시퀀스를 처리할 수 있는 인공 신경망이 필요
* 순환 신경망(Recurrent Neural Network, RNN) 
  * 자연어 처리에 대표적으로 사용되는 인공 신경망
    * 재귀 신경망(Recursive Neural Network)은 전혀 다른 개념
  * 입력과 출력을 시퀀스 단위로 처리하는 **시퀀스(Sequence) 모델**
    * 입력 시퀀스 => 출력 시퀀스 (번역기)
  * RNN 모델
    * 기본적인 순환 신경망인 바닐라 RNN
    * 기본을 개선한 LSTM, GRU

### 1.1. 순환 신경망(Recurrent Neural Network)

![img](https://wikidocs.net/images/page/22886/rnn_image1_ver2.PNG)
* 순방향 신경망과의 비교
  * FFNN: 은닉층에서 활성화 함수의 결과를 출력층으로 보냄
  * RNN: 은닉층에서 활성화 함수의 결과를 **출력층** 방향으로도, 다시 은닉층 노드의 다음 계산의 **입력**으로 보냄

* RNN의 구성
  * 입력 벡터 $x$
  * 출력 벡터 $y$
  * 셀
    * 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드
    * 이전의 값을 기억하려고 하는 일종의 메모리 역할도 수행(RNN셀/메모리셀)


은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 하고 있습니다. 앞으로는 현재 시점을 변수 t로 표현하겠습니다. 이는 현재 시점 t에서의 메모리 셀이 갖고있는 값은 과거의 메모리 셀들의 값에 영향을 받은 것임을 의미합니다. 그렇다면 메모리 셀이 갖고 있는 이 값은 뭐라고 부를까요?

메모리 셀이 출력층 방향 또는 다음 시점인 t+1의 자신에게 보내는 값을 은닉 상태(hidden state) 라고 합니다. 다시 말해 t 시점의 메모리 셀은 t-1 시점의 메모리 셀이 보낸 은닉 상태값을 t 시점의 은닉 상태 계산을 위한 입력값으로 사용합니다.

### 1.2. 케라스(Keras)로 RNN 구현하기

### 1.3. 파이썬으로 RNN 구현하기

### 1.4. 깊은 순환 신경망(Deep Recurrent Neural Network)

### 1.5. 양방향 순환 신경망(Bidirectional Recurrent Neural Network)

### 1.6. 점검 퀴즈

---
## 2. 장단기 메모리(Long Short-Term Memory, LSTM)

### 2.1. 바닐라 RNN의 한계

### 2.2. 바닐라 RNN 내부 열어보기

### 2.3. LSTM(Long Short-Term Memory)

